In [8]:
import qiskit
print("qiskit version:", qiskit.__version__)

qiskit version: 1.4.0


In [9]:
#from qiskit import IBMQ

# Save your IBM Quantum API token
#IBMQ.save_account('a80d2cab0d6db0c274e192f348e18e6c40499d0f97008669e43bfb5db60e9084815b9cd4c8f06740e565a04e6e5a0fc09c39566a8fd79abe309f4ad77bedf432')

In [40]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2
import numpy as np

# Function to create phase oracle
def phase_oracle(n, indices_to_mark, name='Oracle'):
    qc = QuantumCircuit(n, name=name)
    oracle_matrix = np.identity(2**n)
    for index_to_mark in indices_to_mark:
        oracle_matrix[index_to_mark, index_to_mark] = -1
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc

# Optimization: using a dictionary to save created diffusers
diffuser_dict = {}
def diffuser(n):
    if n not in diffuser_dict:
        qc = QuantumCircuit(n, name='Diffuser')
        qc.h(range(n))
        qc.append(phase_oracle(n, [0]), range(n))
        qc.h(range(n))
        diffuser_dict[n] = qc
    return diffuser_dict[n]

# Grover's Algorithm Circuit
def Grover(n, indices_to_mark, r):
    qc = QuantumCircuit(n, n)
    qc.h(range(n))
    qc_oracle = phase_oracle(n, indices_to_mark)
    qc_diffuser = diffuser(n)
    for _ in range(r):
        qc.append(qc_oracle, range(n))
        qc.append(qc_diffuser, range(n))
    qc.measure(range(n), range(n))
    return qc

# Simulating Grover's Algorithm
def run_grover(n, indices_to_mark, r, shots=1024):
    sim = AerSimulator()
    sampler = SamplerV2()
    grover_circuit = Grover(n, indices_to_mark, r)
    grover_circuit = transpile(grover_circuit, sim, optimization_level=0)
    job = sampler.run([(grover_circuit, [])], shots=shots)  # Fixed incorrect sampler input format
    result = job.result()
    print(f"Grover's Algorithm Output: {result[0].data.c.get_counts()}")

# Example: Finding marked states in a 3-qubit system
run_grover(n=3, indices_to_mark=[2], r=1, shots=1024)

Grover's Algorithm Output: {'010': 797, '101': 35, '100': 30, '111': 37, '000': 29, '110': 39, '001': 30, '011': 27}
